In [1]:
import sys
sys.path.insert(0,'C:\\Users\\Syahrir Ridha\\PycharmProjects\\NET_Solver\\')
import numpy as np
import torch
from geometry import *
from utils import Plot_Grid
from hard_boundary import *
from models import *
from mesh import *
from boundary import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nx, ny = 10, 10
anulus = Analytical_Annulus(1., 0.6, 0.)

In [3]:
# define the mesh
xi_m, eta_m = np.linspace(0,1,nx), np.linspace(0,1,ny)
mesh = Hard_Mesh({'xi':xi_m, 'eta':eta_m})


In [4]:
inputs = mesh.gen_data

In [5]:
net = MLP(2,1,10,1,act=torch.nn.Tanh())

In [6]:
trial = Trial_Solution(net, nx, ny)

In [7]:
trial_in = trial(inputs)

In [8]:
def compute_grad(outputs, inputs):
    gradient, = torch.autograd.grad(outputs, inputs, grad_outputs=outputs.data.new(outputs.shape).fill_(1),
                                        create_graph=True, retain_graph=True)
    return gradient

In [9]:
# seperate the inputs
xi, eta = inputs[:,0], inputs[:,1]
        
# calculate the x and y coordinates
grid = TFI(xi, eta, anulus)
x, y = grid.X(), grid.Y()

In [10]:
def cal_grads(x, y, xi, eta):
        # compute all the gradients wrt x,y 
    dxdxi = compute_grad(x, xi)
    dxdeta = compute_grad(x, eta)
    dydxi = compute_grad(y, xi)
    dydeta = compute_grad(y, eta)
    jac = dxdxi*dydeta - dxdeta*dydxi
        
    return dxdxi, dxdeta, dydxi, dydeta, jac

In [11]:
dxdxi, dxdeta, dydxi, dydeta, jac = cal_grads(x, y, xi, eta)

In [12]:
alpha = dxdeta**2 + dydeta**2
gamma = dxdeta*dxdxi + dydeta*dydxi
beta = dxdxi**2 + dydxi**2

In [13]:
out_grad = compute_grad(trial_in, inputs)
dudxi, dudeta = out_grad[:,0], out_grad[:,1]

In [21]:
def shear_rate(out_grad, x, y, xi, eta, tol=1e-5):
    dxdxi, dxdeta, dydxi, dydeta, jac = cal_grads(x, y, xi, eta)
        #u_grad = self.compute_grad(outputs, inputs)
    dudxi, dudeta = out_grad[:,0], out_grad[:,1]
    
    #if tol == None:
        # calculate shear rate
    shear_ = (1/jac)*((dydeta*(dudxi) - dydxi*(dudeta))**2 + (dxdxi*(dudeta) - dxdeta*(dudxi))**2)**(0.5)
    
    # create mask tol_
    tol_ = torch.zeros_like(shear_)
    mask = shear_<=0
    indices = torch.nonzero(mask).unsqueeze(1)
    tol_[indices] = tol
    shear = (1/jac)*((dydeta*(dudxi+tol_) - dydxi*(dudeta+tol_))**2 + (dxdxi*(dudeta+tol_) - dxdeta*(dudxi+tol_))**2)**(0.5)
        
    return shear

In [49]:
shear = shear_rate(out_grad, x, y, xi, eta, tol=1e-6)

In [50]:
tau = 0.5
k = 1.4
n = 0.5
app_vis = (tau/shear) + (shear**(n-1))*k

In [51]:
 compute_grad(app_vis, inputs)
#shear

tensor([[-1.2688e+03, -4.6875e-02],
        [-1.4509e+03, -4.6875e-02],
        [-1.6694e+03,  0.0000e+00],
        [-1.9338e+03,  0.0000e+00],
        [-2.2567e+03, -4.6875e-02],
        [-2.6551e+03, -1.5625e-02],
        [-3.1523e+03, -1.5625e-02],
        [-3.7805e+03,  0.0000e+00],
        [-4.5854e+03, -7.8125e-02],
        [-5.6329e+03, -4.6875e-02],
        [-1.2688e+03, -1.0938e-01],
        [ 2.7118e+05,  3.6968e+05],
        [ 2.6855e+05,  3.8191e+05],
        [ 2.6272e+05,  3.9409e+05],
        [ 2.5281e+05,  4.0620e+05],
        [ 2.3761e+05,  4.1823e+05],
        [ 2.1547e+05,  4.3015e+05],
        [ 1.8411e+05,  4.4193e+05],
        [ 1.4029e+05,  4.5351e+05],
        [-5.6329e+03, -4.6875e-02],
        [-1.2688e+03, -3.1250e-02],
        [ 5.6681e+05,  7.6464e+05],
        [ 5.6148e+05,  7.8911e+05],
        [ 5.4972e+05,  8.1349e+05],
        [ 5.2975e+05,  8.3777e+05],
        [ 4.9915e+05,  8.6185e+05],
        [ 4.5463e+05,  8.8572e+05],
        [ 3.9158e+05,  9.093

In [17]:
def app_visc(out_grad, x, y, xi, eta, n, k, tau):
    shear = shear_rate(out_grad, x, y, xi, eta)
    
    # create a dummy viscosity
    vis = torch.zeros_like(shear)
    
    # mask for shear rate less than critical shear rate
    cs = 0.1
    mask_body = shear>cs
    mask_bound = shear <= cs
    
    ind_body = torch.nonzero(mask_body).unsqueeze(1)
    ind_bound = torch.nonzero(mask_bound).unsqueeze(1)
    
    vis[ind_body] = (tau/shear[ind_body]) + ((shear[ind_body])**(n-1))*k
    vis[ind_bound] = tau*(2 - (shear[ind_bound]/cs))/cs + k * ( (2-n)+(n-1)* (shear[ind_bound]/cs))
    return vis

In [18]:
vis = app_visc(out_grad, x, y, xi, eta, n=0.5, k=1.4, tau=0.5)

In [19]:
left = (vis/jac)*(alpha*dudxi - gamma*dudeta)

In [20]:
#dummy = torch.randn(100)+1
dummy = test.reshape(shear.shape)
vis = torch.zeros_like(dummy)
    
 # mask for shear rate less than critical shear rate
cs = 0.1
mask_body = dummy>cs
mask_bound = dummy <= cs
    
ind_body = torch.nonzero(mask_body).unsqueeze(1)
ind_bound = torch.nonzero(mask_bound).unsqueeze(1)
    
vis[ind_body] = (tau/dummy[ind_body]) + ((dummy[ind_body])**(n-1))*k
vis[ind_bound] = tau*(2 - (dummy[ind_bound]/cs))/cs + k * ( (2-n)+(n-1)* (dummy[ind_bound]/cs))


NameError: name 'test' is not defined

In [ ]:
vis

In [ ]:
dummy

In [ ]:
vis

In [ ]:
test = torch.meshgrid(torch.linspace(0,1,10), torch.linspace(0,1,10))[0].reshape(-1,1)